
# Objectives

Fetch data from Kaggle and prepare it for further processes.

# Inputs

Kaggle JSON file - the authentication token.

Outputs

# Generate Dataset: inputs/datasets/malaria_dataset

Additional Comments | Insights | Conclusions
No additional comments.

In [28]:
%pip install -r/workspaces/Malaria-Detection/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [29]:
import numpy
import os

## Change the working directory

In [30]:
current_dir = os.getcwd()
current_dir

'/workspaces/Malaria-Detection/jupyter_notebooks'

In [31]:
current_dir = os.getcwd()
current_dir

'/workspaces/Malaria-Detection/jupyter_notebooks'

# Install Keggle

In [37]:
%pip install kaggle==1.5.12

Note: you may need to restart the kernel to use updated packages.


In [38]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/workspaces/Malaria-Detection'
!chmod 600 /workspaces/Malaria-Detection/kaggle.json

#### Downloading dataset

In [39]:
KaggleDatasetPath = "codeinstitute/malaria-cell-classification"
DestinationFolder = "inputs/malaria_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

malaria-cell-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


#### Unzipp files and delete the zip

In [40]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/malaria-cell-classification.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/malaria-cell-classification.zip')

----

## Data Preparation

----

### Data Cleaning

Check and remove non-image files

In [41]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location)  # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))

In [42]:
remove_non_image_file(my_data_dir='inputs/malaria_dataset/cell_images')

Folder: Uninfected - has image file 5643
Folder: Uninfected - has non-image file 0
Folder: Parasitized - has image file 5643
Folder: Parasitized - has non-image file 0


### Split train validation test set

In [43]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)  # it should get only the folder name
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)

In [44]:
split_train_validation_test_images(my_data_dir=f"inputs/malaria_dataset/cell_images",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )